<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/main/notebooks/data_preprocessing_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
# use gcs fuse to access google cloud storage
# https://stackoverflow.com/a/60450255/5755604
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# mount google cloud stoarge as drive
!mkdir gcs
!gcsfuse thesis-bucket-option-trade-classification gcs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  47867      0 --:--:-- --:--:-- --:--:-- 47867
OK
33 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.41.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
mkdir: cannot create directory ‘gcs’: File exists
2022/10/28 08:25:47.109418 Start gcsfuse/0.41.7 (Go version go1.18.4) for app "" using mount point: /content/gcs
2022/10/28 08:25:47.130550 Opening GCS connection...
2022/10/28 08:25:47.368553 Mounting file system "thesis-bucket-option-trade-classification"...
2022/10/28 08:25:47.371478 File system has been successfully mounted.


In [42]:
import glob

import modin.pandas as pd
import modin.config as cfg
from modin.config import ProgressBar
from distributed import Client
cfg.Engine.put("dask")
ProgressBar.enable()

# import pandas as pd

from tqdm.notebook import tqdm


In [43]:
from google.colab import auth
auth.authenticate_user()


In [44]:
client = Client()

files = glob.glob("./gcs/data/preprocessed/matched_ise_quotes_min_mem_usage_part_*.parquet")

columns = ['UNDERLYING_SYMBOL', 'QUOTE_DATETIME', 'SEQUENCE_NUMBER', 'ROOT',
       'EXPIRATION', 'STRK_PRC', 'OPTION_TYPE', 'TRADE_SIZE', 'TRADE_PRICE',
       'BEST_BID', 'BEST_ASK', 'order_id', 'ask_ex', 'bid_ex', 'bid_size_ex',
       'ask_size_ex', 'date', 'price_all_lead', 'price_all_lag', 'optionid',
       'day_vol', 'price_ex_lead', 'price_ex_lag', 'buy_sell']

df = pd.read_parquet(files, columns=columns)

Perhaps you already have a cluster running?
Hosting the HTTP server on port 39889 instead


In [45]:
df.memory_usage(deep=True).sum()

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

5417914446

In [61]:
df.head().T

,0,1,2,3,4
UNDERLYING_SYMBOL,BRCD,SYMC,SPY,ADRX,ORCL
QUOTE_DATETIME,2005-05-02 09:30:02,2005-05-02 09:30:03,2005-05-02 09:30:03,2005-05-02 09:30:03,2005-05-02 09:30:03
SEQUENCE_NUMBER,72515,65366,65373,79195,95870
ROOT,YNU,SYQ,SWG,QAX,ORQ
EXPIRATION,2006-01-21 00:00:00,2005-06-18 00:00:00,2005-05-21 00:00:00,2005-06-18 00:00:00,2005-12-17 00:00:00
STRK_PRC,2.5,15.0,105.0,25.0,14.0
OPTION_TYPE,C,C,C,C,C
TRADE_SIZE,10,10,50,10,15
TRADE_PRICE,2.05,3.9,11.2,0.2,0.25
BEST_BID,1.9,3.6,11.1,0.0,0.3


In [66]:
# check against some stats from sub panel A.1 in Grauer et al

# trade size
stats_trade_size = df['TRADE_SIZE'].agg(['mean','median','std'])

# moneyness; price underlying / strike
# TODO: Request price for underlyings?

# time to maturity
stats_time_to_maturity = (df['EXPIRATION'] - df['date']).dt.days
stats_time_to_maturity = stats_time_to_maturity.agg(['mean','median','std'])

# no of observations
stats_n = len(df)

# trade_size = quote size; TRADE_SIZE
stats_trades_with_quote_size_bid = df['bid_size_ex'].eq(df['TRADE_SIZE'])
stats_trades_with_quote_size_ask = df['ask_size_ex'].eq(df['TRADE_SIZE'])
# either ask or bid must be equal, but not both (XOR)
stats_trade_with_quote_size = (stats_trades_with_quote_size_bid ^ stats_trades_with_quote_size_ask).sum() / stats_n

# no of buys
stats_buy_trades = df['buy_sell'].ge(0).sum() / stats_n

# underlyings per day
# stats_underlyings_per_day = df.groupby(['UNDERLYING_SYMBOL','date']).count().agg(['mean','median','std'])

In [53]:
stats_trade_size

mean      13.615033
median     4.000000
std       77.752615
Name: TRADE_SIZE, dtype: float64

In [54]:
stats_time_to_maturity

mean      107.286479
median     46.000000
std       150.079427
dtype: float64

In [50]:
stats_n

49203747

In [68]:
# TODO: mismatch Grauer et. al report 22.28 %
stats_trades_with_quote_size

0.10956509064238543

In [51]:
stats_buy_trades

0.4745849741890592